<a href="https://colab.research.google.com/github/theviderlab/computer_vision/blob/main/Faster_R_CNN_con_Detectron2_Backbone_CVNet_Entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Instalación de dependencias
!pip install -q torch torchvision torchaudio
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install -q wandb
!pip install -q kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.2 MB/s eta 0:00:00
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-s95xb5_f
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp

In [2]:
# 2. Definición de rutas y carga de CVNet
from google.colab import drive
import json
import sys
import os
import copy

# Montaje de Google Drive
drive.mount('/content/drive', force_remount=True)

# Cargar CVNet
base_path = "/content/drive/MyDrive/ViderLab/06 - INFORMACIÓN/Capacitación/Master/TFM/Código/"
cvnet_path = os.path.join(base_path, "image_retrieval", "backbones")
assert os.path.isdir(cvnet_path), f"No existe {cvnet_path}"

if cvnet_path not in sys.path:
    sys.path.insert(0, cvnet_path)

from cvnet.cvnet_model import CVNet_Rerank

# Importar librerías
import matplotlib.pyplot as plt
import torch

from detectron2.layers import ShapeSpec
from detectron2.modeling import BACKBONE_REGISTRY
from detectron2.modeling.backbone import Backbone
from detectron2.modeling.backbone.fpn import FPN, LastLevelMaxPool
from detectron2.config import get_cfg
from detectron2.model_zoo import get_config_file
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data import build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.data.transforms import (
    RandomFlip, RandomRotation, RandomBrightness, RandomContrast,
    RandomLighting, RandomCrop
)
from detectron2.utils.events import EventWriter, get_event_storage, TensorboardXWriter

# Paths para checkpoints y datos
pretrained_weights_path = base_path + "assets/weights/CVPR2022_CVNet_R50.pyth"
dataset_path = base_path + "assets/database/open-images"
images_path = "/tmp/open-images"
train_json_path   = os.path.join(images_path, "annotations_touristic_train.json")
val_json_path   = os.path.join(images_path, "annotations_touristic_val.json")
output_dir = base_path + "/assets/weights/faster_r-cnn_cvnet_finetuned_openimages"
dataset_name = "rodrigodazvelasco/open-images-landmark-subset"

Mounted at /content/drive


In [4]:
import wandb

# Autenticación en W&B
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: theviderlab (theviderlab-viderlab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
# Configurar el dataset desde Kaggle
src = os.path.join(base_path, "kaggle.json")
dst_dir = os.path.expanduser("~/.kaggle")
dst = os.path.join(dst_dir, "kaggle.json")

os.makedirs(dst_dir, exist_ok=True)

!cp "{src}" "{dst}"
!chmod 600 "{dst}"

!kaggle datasets download -d $dataset_name -p $images_path --unzip

Dataset URL: https://www.kaggle.com/datasets/rodrigodazvelasco/open-images-landmark-subset
License(s): unknown
404 Client Error: Not Found for url: https://www.kaggle.com/api/v1/datasets/download/rodrigodazvelasco/open-images-landmark-subset?raw=false


In [4]:
# 3. Carga de anotaciones y definición de clases
with open(train_json_path, 'r') as f:
    train_data = json.load(f)
with open(val_json_path, 'r') as f:
    val_data = json.load(f)

thing_classes = [
    "Building", "Castle", "Fountain", "Lighthouse",
    "Sculpture", "Skyscraper", "Tower"
]

In [5]:
# 4. Registro de dataset en Detectron2
DatasetCatalog.register("openimages_touristic_train", lambda: train_data)
MetadataCatalog.get("openimages_touristic_train").set(thing_classes=thing_classes)
DatasetCatalog.register("openimages_touristic_val", lambda: val_data)
MetadataCatalog.get("openimages_touristic_val").set(thing_classes=thing_classes)

namespace(name='openimages_touristic_val',
          thing_classes=['Building',
                         'Castle',
                         'Fountain',
                         'Lighthouse',
                         'Sculpture',
                         'Skyscraper',
                         'Tower'])

In [6]:
# 5. Wrapper de CVNet para FPN
class CVNetBottomUp(Backbone):
    def __init__(self, cvnet):
        super().__init__()
        self.cvnet = cvnet

    def forward(self, x):
        return self.cvnet.extract_backbone_stages(x)

    def output_shape(self):
        return {
            "res2": ShapeSpec(channels=256, stride=4),
            "res3": ShapeSpec(channels=512, stride=8),
            "res4": ShapeSpec(channels=1024, stride=16),
            "res5": ShapeSpec(channels=2048, stride=32),
        }

In [7]:
# 6. Registro de constructor de backbone CVNet+FPN
@BACKBONE_REGISTRY.register()
def build_cvnet_fpn(cfg, input_shape):
    # Cargar CVNet_Rerank y sus pesos preentrenados
    cvnet = CVNet_Rerank(RESNET_DEPTH=50, REDUCTION_DIM=2048)
    checkpoint = torch.load(pretrained_weights_path, map_location='cpu')
    state = checkpoint.get('model_state', checkpoint)
    cvnet.load_state_dict(state, strict=False)
    cvnet = cvnet.cuda()
    # Congelar CVNet
    for p in cvnet.parameters():
        p.requires_grad = False
    cvnet.eval()
    # Construir FPN sobre CVNet
    bottom_up = CVNetBottomUp(cvnet)
    return FPN(
        bottom_up=bottom_up,
        in_features=cfg.MODEL.FPN.IN_FEATURES,
        out_channels=cfg.MODEL.FPN.OUT_CHANNELS,
        top_block=LastLevelMaxPool(),
        fuse_type=cfg.MODEL.FPN.FUSE_TYPE,
    )

In [8]:
# 7. Configuración base de Detectron2 y del modelo
cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.RESNETS.DEPTH = 50
cfg.MODEL.WEIGHTS = ""  # sin pesos COCO
cfg.MODEL.BACKBONE.NAME = "build_cvnet_fpn"
# Configuración de FPN
cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
cfg.MODEL.FPN.OUT_CHANNELS = 256
cfg.MODEL.FPN.FUSE_TYPE = "sum"

In [9]:
# 8. Parámetros de entrenamiento
# Datasets
cfg.DATASETS.TRAIN = ("openimages_touristic_train",)
cfg.DATASETS.TEST  = ("openimages_touristic_val",)

# Número de trabajadores y directorio de salida
cfg.DATALOADER.NUM_WORKERS = 2
cfg.OUTPUT_DIR = output_dir

# Solver y LR
cfg.SOLVER.IMS_PER_BATCH = 32
cfg.SOLVER.BASE_LR = 5e-5 * (cfg.SOLVER.IMS_PER_BATCH / 2)

# Scheduler: Warmup + MultiStepLR
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupMultiStepLR"

# Factor multiplicativo al llegar a cada step
cfg.SOLVER.GAMMA = 0.1
# Parámetros de warm-up
cfg.SOLVER.WARMUP_FACTOR = 0.001    # LR inicial = BASE_LR * WARMUP_FACTOR
cfg.SOLVER.WARMUP_ITERS = 1000      # iters de warmup
cfg.SOLVER.WARMUP_METHOD = "linear" # lineal desde WARMUP_FACTOR hasta 1

# Definimos iteraciones: 5 épocas ≈ 5 * (160000 / batch)
steps_per_epoch = 160000 // cfg.SOLVER.IMS_PER_BATCH
cfg.SOLVER.MAX_ITER = steps_per_epoch * 5  # ≈ 100k iteraciones
# A qué iteraciones aplicar el decay
cfg.SOLVER.STEPS = (int(cfg.SOLVER.MAX_ITER * 0.6), int(cfg.SOLVER.MAX_ITER * 0.8))

# ROI Heads
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(thing_classes)

# Evaluación periódica
cfg.TEST.EVAL_PERIOD = 1000
cfg.SOLVER.CHECKPOINT_PERIOD = 5000

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [13]:
# 9. Realizar augmentations
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    # Sustituye el path de Drive por el local
    dataset_dict["file_name"] = dataset_dict["file_name"].replace(
        dataset_path,
        images_path
    )
    print(dataset_dict["file_name"]) # debug

    # 1. Leer imagen y anotaciones
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    annos = [obj.copy() for obj in dataset_dict.pop("annotations")]

    # 2. Definir augmentations
    aug_list = [
        RandomFlip(horizontal=True, vertical=False),
        RandomRotation(angle=[-10, 10], sample_style="range"),
        RandomBrightness(0.8, 1.2),
        RandomContrast(0.8, 1.2),
        RandomLighting(0.2),
        RandomCrop("relative_range", (0.8, 0.8)),
    ]

    # 3. Aplicar las transformaciones
    aug_input = T.AugInput(image)
    transforms = T.AugmentationList(aug_list)(aug_input)
    image = aug_input.image

    # 4. Transformar las anotaciones (bboxes)
    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in annos
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])

    # 5. Empaquetar
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
    dataset_dict["instances"] = instances
    return dataset_dict

In [19]:
# 10. Inicializar trainer y entrenar
# — 1) Antes de crear el Trainer, inicializa tu run de W&B:
#    - Asegúrate de estar logueado con wandb.login()
#    - Verifica tu identidad y entidad con: !wandb whoami
#    - Si no perteneces a "theviderlab", omite o cambia `entity=...` al tuyo.
run = wandb.init(
    project="fasterrcnn-cvnet",
    #entity="theviderlab",
    config=cfg,
)

# — 2) Define tu WandbWriter que **usa** el run ya abierto:
class WandbWriter(EventWriter):
    def __init__(self):
        # Si ya hay un run abierto, lo recoge; si no, error
        if wandb.run is None:
            raise RuntimeError("Debes llamar a wandb.init() antes de crear el Trainer")
        self.run = wandb.run

    def write(self):
        storage = get_event_storage()
        latest = storage.latest()
        wandb.log({k: float(v) for k, v in latest.items()}, step=storage.iter)

    def close(self):
        # No cerramos aquí: se cerrará al final de tu script con run.finish()
        pass

# — 3) Subclasa el Trainer para sustituir TensorboardXWriter por tu WandbWriter
class WandbTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper, num_workers=cfg.DATALOADER.NUM_WORKERS)

    @classmethod
    def build_writers(cls, cfg):
        writers = super().build_writers(cfg)
        writers = [w for w in writers if not isinstance(w, TensorboardXWriter)]
        writers.append(WandbWriter())
        return writers

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        from detectron2.evaluation import COCOEvaluator
        return COCOEvaluator(dataset_name, cfg, True, output_folder or os.path.join(cfg.OUTPUT_DIR, "inference"))

# — 4) Úsalo exactamente igual:
trainer = WandbTrainer(cfg)
trainer.build_train_loader = lambda: build_detection_train_loader(cfg, mapper=custom_mapper)
trainer.resume_or_load(resume=False)
trainer.train()

# — 5) Finalmente, cierra el run:
run.finish()

[05/11 10:18:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): CVNetBottomUp(
      (cvnet): CVNet_Rerank(
        (encoder_q): ResNet(
          (stem): ResStemIN(
            (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            (bn): BatchNorm2d(

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


ERROR [05/11 10:20:49 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/detectron2/engine/train_loop.py", line 155, in train
    self.run_step()
  File "/usr/local/lib/python3.11/dist-packages/detectron2/engine/defaults.py", line 530, in run_step
    self._trainer.run_step()
  File "/usr/local/lib/python3.11/dist-packages/detectron2/engine/train_loop.py", line 297, in run_step
    data = next(self._data_loader_iter)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/detectron2/data/common.py", line 329, in __iter__
    for d in self.dataset:
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 708, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1480, in _next_data
    return self._process_data(data)
           ^^^^^^^^^^^^^^^^^^

OSError: Caught OSError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 33, in fetch
    data.append(next(self.dataset_iter))
                ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/detectron2/data/common.py", line 296, in __iter__
    yield self.dataset[idx]
          ~~~~~~~~~~~~^^^^^
  File "/usr/local/lib/python3.11/dist-packages/detectron2/data/common.py", line 125, in __getitem__
    data = self._map_func(self._dataset[cur_idx])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/detectron2/utils/serialize.py", line 26, in __call__
    return self._obj(*args, **kwargs)
           ^^^

In [ ]:
# 11. Lectura y graficado de métricas de entrenamiento
metrics_path = os.path.join(cfg.OUTPUT_DIR, "metrics.json")
metrics = []
with open(metrics_path) as f:
    for line in f:
        if line.strip() and not line.startswith("{\"iteration\": 0"):
            metrics.append(json.loads(line))
iterations = [m["iteration"] for m in metrics]
keys = ["total_loss", "loss_cls", "loss_box_reg", "loss_rpn_cls", "loss_rpn_loc"]
plt.figure(figsize=(12, 6))
for key in keys:
    plt.plot(iterations, [m.get(key) for m in metrics], label=key)
plt.xlabel("Iteración")
plt.ylabel("Pérdida")
plt.title("Curvas de entrenamiento")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()